## Imports

In [1]:
!pip install catboost
!pip install scipy==1.14.0
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 13.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import json
import scipy
from scipy.optimize import nnls
from scipy.optimize import lsq_linear

import ast

import torch
import matplotlib.pyplot as plt

import torch
from tqdm import tqdm
import optuna

from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error, explained_variance_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
vacancies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cost_of_skills/IT_vacancies_full.csv')

In [ ]:
vacancies = vacancies.rename(columns={'Keys': 'Skills'})
vacancies['Skills'] = vacancies['Skills'].apply(lambda x: ast.literal_eval(x))
vacancies['Profarea names'] = vacancies['Profarea names'].apply(lambda x: eval(x))
vacancies['Professional roles'] = vacancies['Professional roles'].apply(lambda x: eval(x))

vacancies['Specializations'] = vacancies['Specializations'].apply(lambda x: eval(x))

In [ ]:
# Applying the logic
vacancies['To'] = vacancies.apply(lambda row: row['From'] if pd.isna(row['To']) and not pd.isna(row['From']) and row['To'] != 0 else row['To'], axis=1)
vacancies['From'] = vacancies.apply(lambda row: row['To'] - 10000 if pd.isna(row['From']) and not pd.isna(row['To']) and row['To'] != 0 else row['From'], axis=1)

# Dropping rows where both From and To are NaN
vacancies.dropna(subset=['From', 'To'], how='all', inplace=True)

vacancies = vacancies[vacancies['To'] != 0]

vacancies = vacancies[vacancies['Salary'] == True]
vacancies = vacancies.drop('Salary', axis=1)

vacancies = vacancies.reset_index()

## Functions

In [ ]:
################################################################################
# Обработка датафрейма и построение матрицы навыков
################################################################################

def get_all_skills(data: pd.DataFrame) -> list:
    """
    Собирает все уникальные навыки из столбца 'Skills' (список навыков в каждой строке).
    Возвращает список навыков (строки в нижнем регистре).
    """
    all_skills = []
    for row in data['Skills']:
        # row - это список навыков для строки датафрейма
        lowercase_list = [item.lower() for item in row]
        all_skills.extend(lowercase_list)

    return list(set(all_skills))  # set() чтобы убрать дубликаты, list() для явного возвращения списка


def make_cf(indices: list, skill_amount: int) -> list:
    """
    Создаёт вектор признаков для одной строки датафрейма.
    Напр. если у строки навыки: [0, 3, 5], а всего навыков skill_amount=10,
    то возвращаем вектор длины 10, где на позициях (0,3,5) будет 1, остальные 0.
    """
    cf = [0]*skill_amount
    for i in indices:
        cf[i] = 1
    return cf


def get_matrix(data: pd.DataFrame, skill_id_dict: dict) -> list:
    """
    Строит матрицу A размера [num_rows x skill_amount],
    где num_rows = число строк в data.
    """
    skill_amount = len(skill_id_dict)
    A = []
    for row in data['Skills']:
        # Преобразуем навыки строки в индексы
        indices = [skill_id_dict[skill.lower()] for skill in row]
        A.append(make_cf(indices, skill_amount))
    return A


################################################################################
# Основная PyTorch-функция, которая оптимизирует интервальный лосс
################################################################################

def calc_torch_interval(A, from_col, to_col, lr=5.0, num_iterations=10000):
    """
    A: матрица признаков (list of lists или numpy), размер [N x M].
    from_col: список или массив c нижней границей зарплаты (длина N).
    to_col: список или массив c верхней границей зарплаты (длина N).
    lr: learning rate.
    num_iterations: количество итераций для оптимизации.

    Возвращает:
      - x: тензор PyTorch (веса навыков),
      - final_interval_loss: скаляр (итоговый интервальный лосс),
      - mae: скаляр (MAE относительно середины интервала).
    """
    # Переводим A и интервалы из Python-списков/NumPy-массивов в PyTorch-тензоры
    A_torch = torch.tensor(A, dtype=torch.float32, device=device)
    from_torch = torch.tensor(from_col, dtype=torch.float32, device=device)
    to_torch = torch.tensor(to_col, dtype=torch.float32, device=device)

    # Параметры модели: вектор x размером M (число навыков)
    # Инициализируем значением 10000.0 (или любым другим)
    x = torch.nn.Parameter(torch.full((A_torch.shape[1],), 10000.0, device=device))

    optimizer = torch.optim.Adam([x], lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.9, patience=500
    )

    for iteration in range(num_iterations):
        optimizer.zero_grad()

        # Предсказанная зарплата для каждой строки: y_pred = A*x
        y_pred = A_torch.matmul(x)

        # Интервальный лосс:
        # loss = среднее (max(0, from_i - y_pred_i) + max(0, y_pred_i - to_i))
        under = torch.clamp(from_torch - y_pred, min=0.0)  # штраф, если y_pred < from
        over  = torch.clamp(y_pred - to_torch, min=0.0)    # штраф, если y_pred > to
        interval_loss = torch.mean(under + over)

        #print(f"Loss {interval_loss}")

        interval_loss.backward()
        optimizer.step()
        scheduler.step(interval_loss)

    # После обучения считаем финальные предсказания
    y_pred_final = A_torch.matmul(x)

    # Финальный интервальный лосс
    final_interval_loss = interval_loss.detach().cpu().item()

    # Средняя точка интервала для каждой строки
    mid_interval = 0.5 * (from_torch + to_torch)

    # MAE относительно средней точки [from, to]
    mae = torch.mean(torch.abs(y_pred_final - mid_interval)).detach().cpu().item()

    return x, final_interval_loss, mae



In [ ]:
#собираем все Profarea names
prof_lst = []
for i in vacancies['Profarea names']:
    prof_lst += i
prof_dict = {i: [] for i in set(prof_lst)}

for i, j in enumerate(vacancies['Profarea names']):
    for k in j:
        prof_dict[k].append(i)

prof = list(prof_dict.keys())

In [ ]:
print(f"Всего категорий {len(prof)}")

Всего категорий 28


In [ ]:
dic_of_mae = {}
for i in prof:
  data =  vacancies[vacancies['Profarea names'].apply(lambda x: any(i in profarea for profarea in x))].copy().dropna().reset_index()

  data = data[['Skills', 'From', 'To']]
  data = data.rename(columns={'From': 'from', 'To': 'to'})

  data = data.dropna()
  data = data[data['from'] != data['to']]
  data = data[data['Skills'].apply(len) > 0]


  # 1. Собираем полный список навыков:
  all_skills = get_all_skills(data)

  # 2. Создаём словарь маппинга навыка -> индекс:
  skill_id_dict = {skill: idx for idx, skill in enumerate(all_skills)}

  # 3. Собираем матрицу A:
  A = get_matrix(data, skill_id_dict)
  x, final_interval_loss, mae = calc_torch_interval(A, data['from'].values, data['to'].values, lr=5, num_iterations=50000)
  #mae_value = calculate_mae_for_profarea(vacancies, i)
  dic_of_mae[i] = final_interval_loss
  print(f"{i} -> {final_interval_loss}")

Юристы -> 0.0
Административный персонал -> 4.806629657745361
Маркетинг, реклама, PR -> 5745.75341796875
Медицина, фармацевтика -> 0.0
Безопасность -> 540.6080932617188
Страхование -> 0.0
Рабочий персонал -> 2029.4794921875
Продажи -> 4778.6220703125
Высший менеджмент -> 0.0
Консультирование -> 255.43357849121094
Наука, образование -> 0.0
Государственная служба, некоммерческие организации -> 4479.16650390625
Бухгалтерия, управленческий учет, финансы предприятия -> 294.1176452636719
Добыча сырья -> 6214.28564453125
Производство, сельское хозяйство -> 1153.3856201171875
Инсталляция и сервис -> 1445.2930908203125
Спортивные клубы, фитнес, салоны красоты -> 0.0
Искусство, развлечения, масс-медиа -> 3069.28759765625
Начало карьеры, студенты -> 468.55230712890625
Информационные технологии, интернет, телеком -> 24475.142578125
Закупки -> 714.2857055664062
Автомобильный бизнес -> 0.0
Строительство, недвижимость -> 106.95187377929688
Транспорт, логистика -> 0.0
Управление персоналом, тренинги ->

In [ ]:
data =  vacancies.copy().dropna().reset_index()
data = data[['Skills', 'From', 'To']]
data = data.rename(columns={'From': 'from', 'To': 'to'})

data = data.dropna()
data = data[data['from'] != data['to']]
data = data[data['Skills'].apply(len) > 0]


# 1. Собираем полный список навыков:
all_skills = get_all_skills(data)

# 2. Создаём словарь маппинга навыка -> индекс:
skill_id_dict = {skill: idx for idx, skill in enumerate(all_skills)}

# 3. Собираем матрицу A:
A = get_matrix(data, skill_id_dict)
x, final_interval_loss, mae = calc_torch_interval(A, data['from'].values, data['to'].values, lr=5, num_iterations=50000)
#mae_value = calculate_mae_for_profarea(vacancies, i)
#dic_of_mae[i] = final_interval_loss
print(f"Loss -> {final_interval_loss}")

Loss -> 24472.212890625


## Catboost

In [ ]:
data =  vacancies.copy().dropna().reset_index()
data = data[['Skills', 'From', 'To']]
data = data.rename(columns={'From': 'from', 'To': 'to'})

data = data.dropna()
data = data[data['from'] != data['to']]
data = data[data['Skills'].apply(len) > 0]


# 1. Собираем полный список навыков:
all_skills = get_all_skills(data)

# 2. Создаём словарь маппинга навыка -> индекс:
skill_id_dict = {skill: idx for idx, skill in enumerate(all_skills)}

# 3. Собираем матрицу A:
A = get_matrix(data, skill_id_dict)

In [ ]:
from catboost import CatBoostRegressor
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Функция для обучения модели CatBoost и вычисления метрик

def calc_catboost_interval(A, from_col, to_col, num_iterations=10000):
    """
    A: матрица признаков (list of lists или numpy), размер [N x M].
    from_col: список или массив с нижней границей зарплаты (длина N).
    to_col: список или массив с верхней границей зарплаты (длина N).
    num_iterations: количество итераций для оптимизации.

    Возвращает:
      - x: numpy массив (веса навыков),
      - final_interval_loss: скаляр (итоговый интервальный лосс),
      - mae: скаляр (MAE относительно средней точки интервала).
    """
    # Преобразование данных в NumPy
    A = np.array(A)
    from_col = np.array(from_col)
    to_col = np.array(to_col)

    # Средняя точка интервала (целевая переменная для обучения)
    mid_interval = 0.5 * (from_col + to_col)

    # Обучение модели CatBoost с интервальным таргетом
    model = CatBoostRegressor(iterations=num_iterations, learning_rate=0.05, depth=6, loss_function='MultiRMSE', verbose=0)
    model.fit(A, np.column_stack((from_col, to_col)))

    # Предсказания модели
    y_pred = model.predict(A)

    # Предсказания средней точки интервала для оценки метрик
    y_pred_mid = 0.5 * (y_pred[:, 0] + y_pred[:, 1])

    # Интервальный лосс
    under = np.maximum(0, from_col - y_pred[:, 0])  # штраф, если y_pred ниже from
    over = np.maximum(0, y_pred[:, 1] - to_col)    # штраф, если y_pred выше to
    final_interval_loss = np.mean(under + over)

    # MAE относительно средней точки интервала
    mae = mean_absolute_error(mid_interval, y_pred_mid)

    # Решение задачи невязки для нахождения весов x
    x, _ = np.linalg.lstsq(A, y_pred_mid, rcond=None)[:2]

    return x, final_interval_loss, mae

# Пример вызова функции
# A - матрица признаков, from_col и to_col - интервалы зарплат
# Пример данных:
A = [[1, 0, 1], [0, 1, 0], [1, 1, 0]]
from_col = [50, 70, 90]
to_col = [80, 100, 120]

x, final_interval_loss, mae = calc_catboost_interval(A, from_col, to_col, num_iterations=10000)
print("Feature Weights (x):", x)
print("Final Interval Loss:", final_interval_loss)
print("MAE:", mae)

Feature Weights (x): [20. 85. 45.]
Final Interval Loss: 4.3797854232252575e-11
MAE: 4.8293221273828145e-11
